# Fidelity calculation using QuTIP

In [25]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import qcd
from qcd.configurations.configuration import ChannelConfiguration
from qcd.configurations import OneShotConfiguration, OneShotEntangledConfiguration
import numpy as np
import math
from typing import Optional, Tuple, cast, List, Dict
from qutip import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.quantum_info.states.utils import partial_trace
from qiskit.result.result import Result
import pickle
import itertools

## Variables to control

In [26]:
# Filenames with results for 3 etas and their optimized parameters to LOAD
# filename = '20210416a_C2_A2_500_10000'  
filename = '20210417_C2_A2_100_100_3_6_20'
#filename = '20210413a_C2_A3_500_10000'  # 2 RX + 2 RY gates

## Auxiliary functions

In [27]:
""" save and load results to and from a file """
def save_results_to_disk(obj, name ):
    with open('results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_results_from_file(name, path):
    with open(path + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [28]:
def _prepare_initial_state_entangled(state_probability: float) -> Tuple[complex, complex, complex, complex]:
    """ Prepare initial state: computing 'y' as the amplitudes  """
    return (0, np.sqrt(state_probability), np.sqrt(1 - state_probability), 0)


In [29]:
def _run_all_circuits_and_return_density_matrices(configuration: ChannelConfiguration):
    """ Create a pair of Quantum Circuits, in its transpiled form, from a given configuration """

    state_vectors = [cast(Result, execute(_create_one_circuit(configuration, eta),
                                         backend=GLOBAL_BACKEND).result()).get_statevector()
                    for eta in configuration.eta_group]

    # print(state_vectors)
    d_m = [[], []] 
    d_m[0] = partial_trace(np.outer(state_vectors[0],state_vectors[0]), [2])  
    d_m[1] = partial_trace(np.outer(state_vectors[1],state_vectors[1]), [2])  
    # print(type(d_m[0]))
    return d_m


In [30]:
def _create_one_circuit(configuration: ChannelConfiguration,
                        eta: float) -> QuantumCircuit:
    """ Creates one circuit from a given  configuration and eta """
    configuration = cast(OneShotEntangledConfiguration, configuration)
    qreg_q = QuantumRegister(3, 'q')
    creg_c = ClassicalRegister(2, 'c')

    initial_state = _prepare_initial_state_entangled(configuration.state_probability)

    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.initialize(initial_state, [0, 1])
    circuit.reset(qreg_q[2])
    circuit.cry(2 * eta, qreg_q[1], qreg_q[2])
    circuit.cx(qreg_q[2], qreg_q[1])
    circuit.cx(qreg_q[0], qreg_q[1])
    circuit.h(qreg_q[0])
    circuit.rx(configuration.angle_rx1, qreg_q[1])
    circuit.ry(configuration.angle_ry1, qreg_q[1])
    circuit.rx(configuration.angle_rx0, qreg_q[0])
    circuit.ry(configuration.angle_ry0, qreg_q[0])
    circuit.measure([0, 1], creg_c)
    return circuit

## Fidelity calculations
At this point only pairs of etas are considered.


In [31]:
trio_results = load_results_from_file(name=filename, path='./results/')

In [40]:
GLOBAL_BACKEND=Aer.get_backend('statevector_simulator')
GLOBAL_ETA_TRIOS = trio_results['eta_groups']
POVM_configurations = trio_results['configurations']

fidelity_list = []
fidelity_results = trio_results
for idx, POVM_configuration in enumerate(POVM_configurations):
    density_matrices = _run_all_circuits_and_return_density_matrices(POVM_configuration)
    fidelity_list.append(fidelity(Qobj(density_matrices[0].data), Qobj(density_matrices[1].data)))


In [33]:
len(fidelity_list)

120

In [41]:
fidelity_list

[0.9965773356759643,
 0.0,
 0.0,
 0.9909427005045196,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9999999999999999,
 0.9753558572841665,
 0.990067226933908,
 0.0,
 0.9933553724254912,
 0.9931504686637359,
 0.9128245063806163,
 0.0,
 0.978659006920883,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8873272025604433,
 0.9656498052522875,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9832457464179412,
 0.9188365810307864,
 0.0,
 0.7773262400220856,
 0.0,
 0.9864247799411142,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6741045405939171,
 0.0,
 0.9354143466934852,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9702609032417313,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.8830571713687072,
 0.9766980572072038,
 0.0,
 0.0,
 0.0,
 0.766044443118978,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9730448705798236,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9999999999999998,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8866206608974704,
 0.9396

In [34]:
def upper_bound_fidelity(fidelity: float) -> float:
    return np.sqrt(1/3) * fidelity

def lower_bound_fidelity(fidelity: float) -> float:
    return (1/2) * (1/3) * fidelity**2

In [35]:
def compute_upper_and_lower_fidelity_bounds(fidelity_list: List[float])-> List[dict]:
    return { 'upper_bound_fidelity': [upper_bound_fidelity(fidelity) for fidelity in fidelity_list],
             'lower_bound_fidelity': [lower_bound_fidelity(fidelity) for fidelity in fidelity_list]}

In [36]:
upper_lower_fidelity_bounds = compute_upper_and_lower_fidelity_bounds(fidelity_list=fidelity_list)

In [37]:
upper_lower_fidelity_bounds['upper_bound_fidelity']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.27895197902735114,
 0.0,
 0.5773502691896257,
 0.0,
 0.5200058729509688,
 0.0,
 0.0,
 0.0,
 0.5708240122350664,
 0.0,
 0.5733956904288109,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5176671401231573,
 0.0,
 0.4461090002655472,
 0.5742170071725621,
 0.0,
 0.0,
 0.5410474827466862,
 0.0,
 0.0,
 0.0,
 0.5771387434054764,
 0.0,
 0.5773502691896257,
 0.0,
 0.5676771963739528,
 0.5583342221802748,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5144542253283655,
 0.0,
 0.0,
 0.0,
 0.42389170868344883,
 0.0,
 0.5471116551071209,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.49093815325361395,
 0.0,
 0.4605541964572831,
 0.0,
 0.0,
 0.0,
 0.4419170347601083,
 0.0,
 0.0,
 0.5773502691896256,
 0.0,
 0.0,
 0.0,
 0.5773502691896257,
 0.0,
 0.0,
 0.5773502691896256,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5773502691896257,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.4422759654459587,
 0.0,
 0.5485684229871858,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5766088

In [38]:
upper_lower_fidelity_bounds['lower_bound_fidelity']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.03890710330163788,
 0.0,
 0.16666666666666666,
 0.0,
 0.13520305395174959,
 0.0,
 0.0,
 0.0,
 0.1629200264720696,
 0.0,
 0.16439130890116638,
 0.0,
 0.0,
 0.0,
 0.0,
 0.13398963398164426,
 0.0,
 0.099506620058963,
 0.16486258566310713,
 0.0,
 0.0,
 0.14636618929326287,
 0.0,
 0.0,
 0.0,
 0.1665445645698262,
 0.0,
 0.16666666666666666,
 0.0,
 0.1611286996414957,
 0.15586855182882625,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.13233157497910433,
 0.0,
 0.0,
 0.0,
 0.08984209034528692,
 0.0,
 0.14966558157702664,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.12051013516003446,
 0.0,
 0.10605508393720688,
 0.0,
 0.0,
 0.0,
 0.09764533280558338,
 0.0,
 0.0,
 0.16666666666666663,
 0.0,
 0.0,
 0.0,
 0.16666666666666666,
 0.0,
 0.0,
 0.16666666666666663,
 0.0,
 0.0,
 0.0,
 0.0,
 0.16666666666666666,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.09780401480557743,
 0.0,
 0.15046365734932401,
 0.0,
 0.0,
 0.0,
 0.0,